## Build-a-thon 2023
Register here https://github.com/vrra/FGAN-Build-a-thon/issues/new/choose

Created: 21 May 2023
Author: Vishnu Ram OV.   
vishnu.n@ieee.org.   
Licence: available for all purposes under the sun  
but with acknowledgement and citation to "ITU FG AN Build-a-thon"

Description: Example notebook for listing , creating and deleting users.
Pre-requisites:
1) user name to be created = e.g. oscar
2) passwd to be set = e.g. secret
3) workspace to be created = e.g. costarica

### Prerequisites
argilla api_url  
argilla api_key  
argilla input_filename  
argilla_workspace  
huggingface hub login token  

### For Colab users:

If you are using colab, add your credentials here and uncomment these  
and skip to step-1 below.

In [ ]:
#api_url="your direct HF Space URL", 
#api_key="your ADMIN_API_KEY"
#input_filename="your input file name"
#argilla_workspace="your argilla workspace"
#my_push_to_hf_hub_token="huggingface hub login token"
#argilla_dbname="argilla db name in hf space"
#hf_hub_name="hf hub name for dataset"
#hf_user="valid hf username where the dataset would be stored"

### For non-colab users:
Create a local file .env with   
api_url="your direct HF Space URL", 
api_key="your ADMIN_API_KEY"  
input_filename="my input filename path"

In [2]:
import argilla as rg

### Read the Argilla HF Space Direct URI, api_key

In [3]:
import os
%pip install python-dotenv
from dotenv import load_dotenv
load_dotenv()

Note: you may need to restart the kernel to use updated packages.


True

In [4]:
uri = os.getenv('api_url')
key = os.getenv('api_key')
my_input_filename = os.getenv('input_filename')
my_argilla_workspace=os.getenv('argilla_workspace')
my_push_to_hf_hub_token=os.getenv('my_push_to_hf_hub_token')
my_argilla_dbname=os.getenv('argilla_dbname')
my_hf_hub_name=os.getenv('hf_hub_name')
my_hf_user=os.getenv('hf_user')

In [5]:
rg.init(    
    api_url=uri, 
    api_key=key
    #workspace=my_argilla_workspace
)
#in this case, we will be creating a new workspace for the user, 
#see below

In [6]:
import httpx
auth_headers = {"X-Argilla-API-Key": key}
http = httpx.Client(base_url=uri, headers=auth_headers)
import json


### List all the users, useful to id the user

In [8]:
users = http.get("/api/users").json()
#Input the username from the admin
print("Enter the username to be created: ")
created_username= input()

for item in users:
    if (created_username == item['username']):
        created_username_id=item['id']
        print("***** User exists: \""+ 
              created_username+ 
              "\" with id= "+
              item['id']+
              " Dont create this user again")


***** User exists: "oscar" with id= a5b47ad4-1bca-4cde-a063-aed8d7a6ee56 Dont create this user again


### List all workspaces

In [118]:
workspaces = http.get("/api/workspaces").json()
#Input the workspace from the admin
print("Enter the workspace to be created: ")
created_workspace= input()

created_workspace_id = -1 #setup an invariant.
for item in workspaces:
    if (created_workspace == item['name']):
        created_workspace_id = item['id']
        print("***** workspace exists: \""+ 
              created_workspace+ 
              "\" with id= "+
              created_workspace_id+ 
              " Dont create this workspace again")

***** workspace exists: "costarica" with id= 0f18d550-9a65-4d7f-a9d2-12b90f4f4dda Dont create this workspace again


### Creating user with the details.
CAUTION: execute this only if the user does not exist already

In [ ]:
created_role="annotator"
created_first_name=created_username
created_last_name= "None"
print("Enter the passwd to be created: ")
created_password= input()

user = http.post("/api/users", 
                 json={"role": created_role, 
                       "first_name": created_first_name, 
                       "last_name": created_last_name, 
                       "username": created_username, 
                       "password": created_password}).json()

print(json.dumps(user, indent = 1))

### Create workspace
CAUTION: execute this only if the workspace doesnt exist already

In [68]:
#created_workspace="costarica"
if (created_workspace_id != -1):
    workspace = http.post("/api/workspaces", json={"name": created_workspace}).json()

### Assign user to workspace

In [119]:
http.post("/api/workspaces/"+
          created_workspace_id+
          "/users/"+
          created_username_id)

<Response [409 Conflict]>

### set the workspace
so that we can import the dataset

In [120]:
rg.set_workspace(created_workspace)

### Import the dataset for the user

In [ ]:
from datasets import load_dataset
load_dataset_path=my_hf_user+'/'+my_hf_hub_name
print(load_dataset_path)
dataset_ds = load_dataset(load_dataset_path, split="train")
dataset_rg = rg.read_datasets(dataset_ds, task="TextClassification")

#the dataset is hardcoded because this is just a test code.
my_db_name="fgandataset_annotate"
rg.log(dataset_rg, my_db_name)

#Now you can go to the HF space and check if you got it.

In [123]:
settings = rg.TextClassificationSettings(label_schema=["Usecase", "Architecture", "PoC", "Other"])
rg.configure_dataset(name=my_db_name, settings=settings)

### CAUTION: Dont do! Deleting user! this needs the id from above.

In [ ]:
print(json.dumps(user, indent = 1))
print(created_username_id)
http.delete("/api/users/"+
            created_username_id
            ).json()

### CAUTION: Dont do! Trying to del the workspace, but this does not work for some reason

In [96]:
http.delete("/api/workspaces/"+created_workspace_id).json()

{'detail': 'Method Not Allowed'}